In [ ]:
Python Code to Connect to Your Database

In [1]:
import pyodbc
import pandas as pd


In [ ]:
# Define connection details

In [2]:
server = r'DESKTOP-39CALVC\SQLEXPRESS'  
database = 'EmployeeAttrition'

In [ ]:
# Trusted connection (Windows Authentication)

In [3]:
conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

In [8]:
from sqlalchemy import create_engine
import pandas as pd


In [ ]:
# Connection string for SQLAlchemy using Windows Authentication

In [9]:
connection_string = (
    'mssql+pyodbc://@DESKTOP-39CALVC\\SQLEXPRESS/EmployeeAttrition'
    '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
)

In [ ]:
# Create the engine

In [10]:
engine = create_engine(connection_string)

In [ ]:
# Example SQL query

In [13]:
query = "SELECT TOP 5 * FROM EmployeeAttrition"

In [ ]:
# Use pandas with SQLAlchemy engine

In [14]:
df = pd.read_sql(query, engine)
df.head()

,Employee_ID,Age,Gender,Marital_Status,Department,Job_Role,Job_Level,Monthly_Income,Hourly_Rate,Years_at_Company,...,Overtime,Project_Count,Average_Hours_Worked_Per_Week,Absenteeism,Work_Environment_Satisfaction,Relationship_with_Manager,Job_Involvement,Distance_From_Home,Number_of_Companies_Worked,Attrition
0,1,58,Male,Single,Finance,Manager,5,7332,81,24,...,No,9,48,16,4,1,1,49,3,No
1,2,48,Female,Divorced,HR,Assistant,4,6069,55,18,...,Yes,9,57,10,4,1,1,25,1,No
2,3,34,Female,Married,Marketing,Manager,4,11485,65,6,...,Yes,3,55,1,1,4,3,21,1,Yes
3,4,27,Female,Divorced,HR,Manager,4,18707,28,12,...,No,9,53,2,3,4,1,46,2,No
4,5,40,Male,Married,HR,Analyst,1,16398,92,3,...,No,1,54,11,1,1,1,43,4,No


In [ ]:
1. Total Number of Employees

In [20]:
 query ="SELECT COUNT(*) AS Total_Employees FROM EmployeeAttrition"

In [21]:
df = pd.read_sql(query, engine)
df.head()

,Total_Employees
0,10000


In [ ]:
2. Overall Attrition Count and Rate

In [26]:
query = """
SELECT 
    COUNT(*) AS Attrition_Count,
    CAST(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM EmployeeAttrition) AS DECIMAL(5,2)) AS Attrition_Rate
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition_Count,Attrition_Rate
0,1997,19.97


In [ ]:
3. Department-wise Attrition Count


In [27]:
query = """
SELECT Department, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Department
"""
df = pd.read_sql(query, engine)
df.head()


,Department,Attrition_Count
0,Finance,415
1,Sales,398
2,Marketing,413
3,HR,381
4,IT,390


In [ ]:
4. Department-wise Attrition Rate

In [28]:
query = """
SELECT Department,
       COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END) * 100.0 / COUNT(*) AS Attrition_Rate
FROM EmployeeAttrition
GROUP BY Department
"""
df = pd.read_sql(query, engine)
df.head()


,Department,Attrition_Rate
0,Finance,20.854271
1,Sales,19.820717
2,Marketing,19.362400
3,HR,19.508449
4,IT,20.354906


In [ ]:
5. Gender-wise Attrition Rate

In [29]:
query = """
SELECT Gender,
       COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END) * 100.0 / COUNT(*) AS Attrition_Rate
FROM EmployeeAttrition
GROUP BY Gender
"""
df = pd.read_sql(query, engine)
df.head()


,Gender,Attrition_Rate
0,Male,20.270270
1,Female,19.674732


In [ ]:
✅ 6. Attrition by Job Role

In [30]:
query = """
SELECT Job_Role, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Job_Role
"""
df = pd.read_sql(query, engine)
df.head()


,Job_Role,Attrition_Count
0,Manager,436
1,Executive,497
2,Assistant,544
3,Analyst,520


In [ ]:
✅ 7. Salary Band-wise Attrition


In [31]:
query = """
SELECT 
    CASE 
        WHEN Monthly_Income < 3000 THEN 'Low'
        WHEN Monthly_Income BETWEEN 3000 AND 7000 THEN 'Medium'
        WHEN Monthly_Income BETWEEN 7001 AND 12000 THEN 'High'
        ELSE 'Very High'
    END AS Salary_Band,
    COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY 
    CASE 
        WHEN Monthly_Income < 3000 THEN 'Low'
        WHEN Monthly_Income BETWEEN 3000 AND 7000 THEN 'Medium'
        WHEN Monthly_Income BETWEEN 7001 AND 12000 THEN 'High'
        ELSE 'Very High'
    END
"""
df = pd.read_sql(query, engine)
df.head()


,Salary_Band,Attrition_Count
0,Very High,915
1,High,616
2,Medium,466


In [ ]:
✅ 8. Avg Years Since Last Promotion by Attrition


In [32]:
query = """
SELECT Attrition, AVG(Years_Since_Last_Promotion) AS Avg_Years_Since_Promotion
FROM EmployeeAttrition
GROUP BY Attrition
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition,Avg_Years_Since_Promotion
0,Yes,4
1,No,4


In [ ]:
✅ 9. Attrition Based on Overtime

In [33]:
query = """
SELECT Overtime,
       COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END) * 100.0 / COUNT(*) AS Attrition_Rate
FROM EmployeeAttrition
GROUP BY Overtime
"""
df = pd.read_sql(query, engine)
df.head()


,Overtime,Attrition_Rate
0,Yes,20.073514
1,No,19.870664


In [ ]:
✅ 10. Avg Job Satisfaction by Attrition

In [34]:
query = """
SELECT Attrition, AVG(Job_Satisfaction) AS Avg_Job_Satisfaction
FROM EmployeeAttrition
GROUP BY Attrition
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition,Avg_Job_Satisfaction
0,Yes,3
1,No,3


In [ ]:
✅ 11. Performance Rating vs. Attrition


In [35]:
query = """
SELECT Performance_Rating, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Performance_Rating
"""
df = pd.read_sql(query, engine)
df.head()


,Performance_Rating,Attrition_Count
0,3,530
1,1,481
2,4,509
3,2,477


In [ ]:
✅ 12. Age Group-wise Attrition



In [36]:
query = """
SELECT 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 45 THEN '30-45'
        ELSE 'Above 45'
    END AS Age_Group,
    COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 45 THEN '30-45'
        ELSE 'Above 45'
    END
"""
df = pd.read_sql(query, engine)
df.head()


,Age_Group,Attrition_Count
0,30-45,798
1,Under 30,483
2,Above 45,716


In [ ]:
✅ 13. Avg Years at Company by Attrition

In [37]:
query = """
SELECT Attrition, AVG(Years_at_Company) AS Avg_Years_At_Company
FROM EmployeeAttrition
GROUP BY Attrition
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition,Avg_Years_At_Company
0,Yes,14
1,No,14


In [ ]:
✅ 14. Avg Absenteeism by Attrition

In [38]:
query = """
SELECT Attrition, AVG(Absenteeism) AS Avg_Absenteeism
FROM EmployeeAttrition
GROUP BY Attrition
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition,Avg_Absenteeism
0,Yes,9
1,No,9


In [ ]:
✅ 15. Attrition Based on Marital Status

In [39]:
query = """
SELECT Marital_Status, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Marital_Status
"""
df = pd.read_sql(query, engine)
df.head()


,Marital_Status,Attrition_Count
0,Married,649
1,Divorced,684
2,Single,664


In [ ]:
✅ 16. Attrition Based on Distance from Home

In [40]:
query = """
SELECT 
    CASE 
        WHEN Distance_From_Home < 5 THEN '0-5 km'
        WHEN Distance_From_Home BETWEEN 5 AND 15 THEN '5-15 km'
        ELSE '15+ km'
    END AS Distance_Band,
    COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY 
    CASE 
        WHEN Distance_From_Home < 5 THEN '0-5 km'
        WHEN Distance_From_Home BETWEEN 5 AND 15 THEN '5-15 km'
        ELSE '15+ km'
    END
"""
df = pd.read_sql(query, engine)
df.head()


,Distance_Band,Attrition_Count
0,15+ km,1406
1,5-15 km,446
2,0-5 km,145


In [ ]:
✅ 17. Job Involvement vs. Attrition

In [41]:
query = """
SELECT Job_Involvement, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Job_Involvement
"""
df = pd.read_sql(query, engine)
df.head()


,Job_Involvement,Attrition_Count
0,3,532
1,1,472
2,4,508
3,2,485


In [ ]:
✅ 18. Attrition by Number of Companies Worked

In [42]:
query = """
SELECT Number_of_Companies_Worked, COUNT(*) AS Attrition_Count
FROM EmployeeAttrition
WHERE Attrition = 'Yes'
GROUP BY Number_of_Companies_Worked
"""
df = pd.read_sql(query, engine)
df.head()


,Number_of_Companies_Worked,Attrition_Count
0,3,502
1,1,483
2,4,488
3,2,524


In [ ]:
✅ 19. Avg Work-Life Balance Score by Attrition

In [43]:
query = """
SELECT Attrition, AVG(Work_Life_Balance) AS Avg_WorkLife_Balance
FROM EmployeeAttrition
GROUP BY Attrition
"""
df = pd.read_sql(query, engine)
df.head()


,Attrition,Avg_WorkLife_Balance
0,Yes,2
1,No,2


In [ ]:
✅ 20. High Risk Employees (Attrition Likely)

In [44]:
query = """
SELECT *
FROM EmployeeAttrition
WHERE 
    Attrition = 'Yes' AND
    Overtime = 'Yes' AND
    Job_Satisfaction < 3 AND
    Years_Since_Last_Promotion >= 3 AND
    Monthly_Income < 5000
"""
df = pd.read_sql(query, engine)
df.head()


,Employee_ID,Age,Gender,Marital_Status,Department,Job_Role,Job_Level,Monthly_Income,Hourly_Rate,Years_at_Company,...,Overtime,Project_Count,Average_Hours_Worked_Per_Week,Absenteeism,Work_Environment_Satisfaction,Relationship_with_Manager,Job_Involvement,Distance_From_Home,Number_of_Companies_Worked,Attrition
0,208,45,Female,Divorced,IT,Analyst,2,4186,60,20,...,Yes,7,33,19,2,2,3,3,4,Yes
1,468,56,Male,Divorced,Marketing,Analyst,2,4144,43,28,...,Yes,8,44,4,4,1,3,35,2,Yes
2,1328,49,Female,Divorced,IT,Assistant,4,4343,17,13,...,Yes,2,36,2,2,4,1,49,1,Yes
3,1503,53,Female,Married,Marketing,Executive,3,4927,70,15,...,Yes,1,45,10,1,4,2,19,2,Yes
4,1515,51,Female,Single,Marketing,Manager,2,3986,88,7,...,Yes,2,32,10,2,4,1,21,2,Yes


In [ ]:
THANK YOU BY DURGAM MANOHAR